In [0]:
!pip install delta-sharing

In [0]:
import delta_sharing

# Point to the profile file. It can be a file on the local file system or a file on a remote storage.
profile_file = "/content/config.share"


In [0]:
# Create a SharingClient.
client = delta_sharing.SharingClient(profile_file)

In [0]:
# List all shared tables.
client.list_all_tables()

In [0]:
# Create a url to access a shared table.
# A table path is the profile file path following with `#` and the fully qualified name of a table
# (`<share-name>.<schema-name>.<table-name>`).
table_url = profile_file + "#share__products.bronze.sample_amazon_reviews"

In [0]:
# Fetch 10 rows from a table and convert it to a Pandas DataFrame. This can be used to read sample data
# from a table that cannot fit in the memory.
# delta_sharing.load_as_pandas(table_url, limit=10)

# Load a table as a Pandas DataFrame. This can be used to process tables that can fit in the memory.
# delta_sharing.load_as_pandas(table_url)

# If the code is running with PySpark, you can use `load_as_spark` to load the table as a Spark DataFrame.
# delta_sharing.load_as_spark(table_url)


In [0]:
table_url

# EDA

### Preguntas
- Las imagenes existen?
- Como podemos analizar que una review sea verificada. Son mas importantes?
-  Existen productos puntuados pero sin reseñas.
- Existe una forma de ver el producto?: Seria interesante poder visualizar el producto que se quiere mejorar.
- *unixReviewTime*: Este tiempo es para la verificación de la review? y este tiempo en que unidades es.

## Primera revisión

La mayoría de imagenes no se encuentra, las imagenes son una url. Seria bueno revisar la metadata para ver si podemos optener las imagenes de los productos. Adicional mente, hay reviews vacios donde solo se tienen las puntuaciones. Sobre *unixReviewTime*, si es el tiempo de verificación, seria bueno saber cuanto es el costo de cuanto demora esa revisión.

In [0]:
data = delta_sharing.load_as_pandas(table_url)
data.head()

In [0]:
import missingno as msno

msno.matrix(data)

In [0]:
data.isnull().sum()

In [0]:
data[data["reviewText"].isnull()].head()

Un "timestamp" es una forma de representar una fecha y hora en la computadora como un número entero, contando los segundos transcurridos desde una fecha de referencia. En este caso, el timestamp 1411948800 representa el número de segundos que han transcurrido desde el 1 de enero de 1970, que es una fecha de referencia comúnmente utilizada en sistemas operativos y lenguajes de programación, y se le conoce como el "Unix Epoch time".

In [0]:
data["unixReviewTime"][0]

In [0]:
import datetime

def timestamp_to_date(timestamp):
    try:
        # Convertir el timestamp a un objeto datetime
        dt_object = datetime.datetime.fromtimestamp(timestamp)

        # Obtener la fecha y hora legible
        formatted_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')
        return formatted_date
    except Exception as e:
        return f"Error al convertir el timestamp: {e}"

# Ejemplo de uso
for i in range(100):
  timestamp = int(data["unixReviewTime"][i])
  fecha_legible = timestamp_to_date(timestamp)
  print(f"Fecha legible: {fecha_legible}")


In [0]:
data["image"][42266]

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
conteo_puntuaciones =  data['overall'].value_counts()

plt.bar(conteo_puntuaciones.index, conteo_puntuaciones.values)
plt.xlabel('Puntuación')
plt.ylabel('Cantidad')
plt.title('Distribución de puntuaciones')
plt.xticks(conteo_puntuaciones.index)
plt.show()

Quiero ver si la verificación agrega alguna particularidad que podamos usar.

## Explorando la verificación True

In [0]:
# Filtrar las filas donde 'verified' es igual a True
data_vtrue =  data[data['verified'] == "true"]
data_vtrue


Con el style podemos saber un poco mas sobre el producto. Por ejemplo hay Elsa Dress, Kindle Edition, Hardcover que probablemente sea un libro.

In [0]:
data_vtrue["style"].head(30)

Veamos algunas reseñas que tienen verificación.

In [0]:
[ str(i)+" Vote:" + str(data_vtrue["vote"][i]) +" " + data_vtrue["reviewText"][i]+" OVERALL: "+data_vtrue["overall"][i] for i in list(data_vtrue.index)[:80]]

Seria bueno un modelo de filtrado para conocer la intención de las reseñas, para así poder ver cuales son las que nos pueden ser utiles para mejorar el negocio.

La reseñas algunas son elogiando el producto, otras son quejas muy especificas sobre daños del producto, lo cual me parece muy util.

In [0]:
conteo_puntuaciones =  data_vtrue['overall'].value_counts()

plt.bar(conteo_puntuaciones.index, conteo_puntuaciones.values)
plt.xlabel('Puntuación')
plt.ylabel('Cantidad')
plt.title('Distribución de puntuaciones con verificacion')
plt.xticks(conteo_puntuaciones.index)
plt.show()

## Explorando la verificacion False

In [0]:
# Filtrar las filas donde 'verified' es igual a True
data_vfalse =  data[data['verified'] == "false"]
data_vfalse


In [0]:
conteo_puntuaciones =  data_vfalse['overall'].value_counts()

plt.bar(conteo_puntuaciones.index, conteo_puntuaciones.values)
plt.xlabel('Puntuación')
plt.ylabel('Cantidad')
plt.title('Distribución de puntuaciones con verificacion')
plt.xticks(conteo_puntuaciones.index)
plt.show()

In [0]:
[ str(i)+" Vote:" + str(data_vfalse["vote"][i]) +" " + str(data_vfalse["reviewText"][i])+" OVERALL: "+str(data_vfalse["overall"][i]) for i in list(data_vfalse.index)[:80]]

No veo ninguna particularidad que diferencia una review verificada y una que no. De hecho, se me hace muy confuso que significa que sea verificada ya que hay unas que son elogios y son verificados. Pensaría que tendría mucho mas sentido si verificara alguna error o algo así. Aunque ahora que lo pienso, puede referirse a la verificación de usuario y eso le daría cierto tipo de confiabilidad pero aún no veo claro si sea muy util esto del verificado.

Al final la verificación no creo que sea algo totalmente sustancial para ayudar la calidad de la ventas, de hecho, que sea verificado o no, no agrega mucho cambio o particularidad.

# Exploremos algunos productos

In [0]:
data

Hay 111866 filas, pero en total hay 2024 productos. Eso quiere decir que hay varios productos que compran seguido.

In [0]:
data['asin'].value_counts()

In [0]:
# Filtremos un producto
p1 =  data[data['asin'] == "B000MXQ2CU"]
p1

In [0]:
p1["style"]

In [0]:
conteo_puntuaciones =  p1['overall'].value_counts()

plt.bar(conteo_puntuaciones.index, conteo_puntuaciones.values)
plt.xlabel('Puntuación')
plt.ylabel('Cantidad')
plt.title('Distribución de puntuaciones con verificacion')
plt.xticks(conteo_puntuaciones.index)
plt.show()

In [0]:
list(p1.index)


In [0]:
import datetime

def timestamp_to_date(timestamp):
    try:
        # Convertir el timestamp a un objeto datetime
        dt_object = datetime.datetime.fromtimestamp(timestamp)

        # Obtener la fecha y hora legible
        formatted_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')
        return formatted_date
    except Exception as e:
        return f"Error al convertir el timestamp: {e}"

# Ejemplo de uso
for i in (list(p1.index)):
  timestamp = int(p1["unixReviewTime"][i])
  fecha_legible = timestamp_to_date(timestamp)
  print(f"Fecha legible: {fecha_legible}")


In [0]:
dict(p1["unixReviewTime"].value_counts())

In [0]:

# Convertir el diccionario a una lista de tuplas (timestamp, valor)
data_tuples = [(int(timestamp), value) for timestamp, value in dict(p1["unixReviewTime"].value_counts()).items()]

# Crear un DataFrame a partir de la lista de tuplas
df = pd.DataFrame(data_tuples, columns=['unix_timestamp', 'value'])

# Función para plotear la gráfica de tiempo
def plot_unix_time(df, timestamp_column):
    df['Datetime'] = pd.to_datetime(df[timestamp_column], unit='s')
    df.set_index('Datetime', inplace=True)

    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['value'])
    plt.xlabel('Fecha')
    plt.ylabel('Valor')
    plt.title('Gráfica de serie de tiempo')
    plt.xticks(rotation=24)
    plt.show()

# Utilizar la función para plotear la gráfica de tiempo
plot_unix_time(df, 'unix_timestamp')